# BLIP-2视觉问答与图像描述教程

> 完整演示BLIP-2模型的各种使用方式

**学习目标**：
- 掌握BLIP-2的图像描述生成
- 学会使用BLIP-2进行视觉问答
- 了解图像-文本相似度计算
- 探索BLIP-2的实际应用

**预计时间**: 40-50分钟

In [ ]:
# 检查并安装依赖
try:
    from transformers import Blip2Processor, Blip2ForConditionalGeneration
    print("✅ transformers已安装")
except ImportError:
    print("正在安装transformers...")
    !pip install transformers
    print("✅ 安装完成")

In [ ]:
# 导入必要的库
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
import os

from transformers import Blip2Processor, Blip2ForConditionalGeneration

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. 加载BLIP-2模型

BLIP-2有多种配置，我们使用`opt-2.7b`版本进行演示。

In [ ]:
# 加载模型和处理器
model_name = "Salesforce/blip2-opt-2.7b"

print(f"📥 加载模型: {model_name}")
print("   (首次运行会下载模型，大约5.5GB，请耐心等待...)")

processor = Blip2Processor.from_pretrained(model_name)
model = Blip2ForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16  # 使用FP16节省显存
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

print(f"✅ 模型加载完成，使用设备: {device}")

## 2. 准备示例图像

In [ ]:
# 准备示例图像
image_path = "sample_image.jpg"

# 尝试下载示例图像
try:
    if not os.path.exists(image_path):
        print("📥 下载示例图像...")
        image_url = "https://images.unsplash.com/photo-1574158622682-e40e69881006?w=400"
        urllib.request.urlretrieve(image_url, image_path)
        print("✅ 下载成功")
except Exception as e:
    print(f"⚠️ 下载失败: {e}")
    print("生成测试图像...")
    # 生成一个简单的测试图像（不依赖OpenCV）
    from PIL import ImageDraw, ImageFont
    
    # 创建渐变背景图像
    test_image = Image.new('RGB', (600, 400))
    pixels = test_image.load()
    
    for i in range(400):
        for j in range(600):
            # 创建渐变效果
            r = int(100 + 155 * (j / 600))
            g = int(150 + 105 * (i / 400))
            b = int(200 - 100 * ((i + j) / 1000))
            pixels[j, i] = (r, g, b)
    
    # 绘制形状和文本
    draw = ImageDraw.Draw(test_image)
    
    # 绘制矩形
    draw.rectangle([100, 100, 500, 300], outline='blue', width=3)
    
    # 绘制圆形
    draw.ellipse([250, 150, 350, 250], outline='red', width=3)
    
    # 添加文本
    try:
        # 尝试使用默认字体
        draw.text((250, 320), "Test Image", fill='white')
    except:
        # 如果没有字体也没关系
        pass
    
    test_image.save(image_path)
    print("✅ 生成测试图像（无需OpenCV）")

# 加载并显示图像
image = Image.open(image_path).convert("RGB")

plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.title("示例图像")
plt.axis('off')
plt.show()

print(f"图像尺寸: {image.size}")

## 3. 图像描述生成 (Image Captioning)

BLIP-2可以自动生成图像的描述。

In [ ]:
def generate_caption(image, prompt=None, max_new_tokens=50):
    """生成图像描述"""
    if prompt:
        inputs = processor(images=image, text=prompt, return_tensors="pt").to(device, torch.float16)
    else:
        inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    
    return caption

# 方式1：无提示（自动描述）
caption = generate_caption(image)
print(f"📝 自动生成的描述:")
print(f"   {caption}")

# 方式2：带提示
caption_detailed = generate_caption(image, prompt="A detailed description:")
print(f"\n📝 详细描述:")
print(f"   {caption_detailed}")

## 4. 视觉问答 (Visual Question Answering)

BLIP-2可以回答关于图像的问题。

In [ ]:
def visual_qa(image, question):
    """视觉问答"""
    prompt = f"Question: {question} Answer:"
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device, torch.float16)
    
    generated_ids = model.generate(**inputs, max_new_tokens=20)
    answer = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    
    # 清理答案
    if answer.startswith(prompt):
        answer = answer[len(prompt):].strip()
    
    return answer

# 测试多个问题
questions = [
    "What is the main subject of this image?",
    "What color is prominent in the image?",
    "Is this taken indoors or outdoors?",
    "What is the mood of the image?"
]

print("❓ 视觉问答示例:\n")
for q in questions:
    answer = visual_qa(image, q)
    print(f"Q: {q}")
    print(f"A: {answer}")
    print()

## 5. 多轮对话（带上下文管理）

演示真正的多轮对话：后续问题可以指代之前的回答。

In [ ]:
def multi_turn_conversation(image, questions):
    """多轮对话（带上下文累积）"""
    conversation = []
    context = ""  # 累积上下文
    
    for question in questions:
        # 构建包含历史的提示
        if context:
            # 包含之前的对话历史
            prompt = f"{context}\nQuestion: {question} Answer:"
        else:
            prompt = f"Question: {question} Answer:"
        
        # 使用带上下文的提示生成答案
        inputs = processor(images=image, text=prompt, return_tensors="pt").to(device, torch.float16)
        generated_ids = model.generate(**inputs, max_new_tokens=30)
        full_response = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        
        # 提取答案部分
        if "Answer:" in full_response:
            answer = full_response.split("Answer:")[-1].strip()
        else:
            answer = full_response
        
        # 记录对话
        conversation.append((question, answer))
        
        # 更新上下文（保留最近3轮对话）
        context_entry = f"Q: {question}\nA: {answer}"
        if context:
            context_parts = context.split("\n\n")
            context_parts.append(context_entry)
            # 只保留最近3轮
            context = "\n\n".join(context_parts[-3:])
        else:
            context = context_entry
    
    return conversation

# 示例对话（问题有连贯性）
conversation_questions = [
    "What is the main subject in this image?",
    "What color is it?",  # 指代前一个问题的主体
    "Based on what we discussed, what might this scene represent?"
]

print("💬 多轮对话（带上下文）:\n")
conversation = multi_turn_conversation(image, conversation_questions)

for i, (q, a) in enumerate(conversation, 1):
    print(f"回合 {i}:")
    print(f"  人类: {q}")
    print(f"  BLIP-2: {a}")
    print()

print("\n💡 注意：多轮对话通过在提示中累积历史Q&A来实现上下文")
print("   这样后续问题可以指代之前讨论的内容")

## 6. 批量处理

演示如何批量处理多张图像。

In [ ]:
# 创建多张测试图像（不依赖OpenCV）
from PIL import ImageDraw

test_images = []
for i in range(3):
    # 生成不同颜色的测试图像
    base_color = (
        100 + i * 50,  # R
        150 + i * 30,  # G
        200 - i * 40   # B
    )
    
    img = Image.new('RGB', (400, 300), base_color)
    draw = ImageDraw.Draw(img)
    
    # 绘制不同的形状
    if i == 0:
        draw.rectangle([50, 50, 350, 250], outline='white', width=5)
    elif i == 1:
        draw.ellipse([50, 50, 350, 250], outline='white', width=5)
    else:
        draw.polygon([(200, 50), (350, 150), (275, 250), (125, 250), (50, 150)], outline='white', width=5)
    
    # 添加标记文本
    draw.text((150, 270), f"Test Image {i+1}", fill='white')
    
    test_images.append(img)

# 显示测试图像
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, img in enumerate(test_images):
    axes[i].imshow(img)
    axes[i].set_title(f"Test Image {i+1}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

# 批量生成描述
print("\n📝 批量生成描述:\n")
for i, img in enumerate(test_images, 1):
    caption = generate_caption(img)
    print(f"Image {i}: {caption}")

## 7. 生成参数调优

探索不同的生成参数如何影响输出。

In [ ]:
def generate_with_params(image, **kwargs):
    """使用自定义参数生成"""
    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs, **kwargs)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    return caption

print("🎛️ 不同生成参数的效果:\n")

# 1. 默认参数
print("1. 默认参数:")
caption1 = generate_with_params(image, max_new_tokens=50)
print(f"   {caption1}\n")

# 2. 束搜索
print("2. 束搜索 (num_beams=5):")
caption2 = generate_with_params(image, max_new_tokens=50, num_beams=5)
print(f"   {caption2}\n")

# 3. 采样
print("3. 随机采样 (do_sample=True, temperature=0.7):")
caption3 = generate_with_params(image, max_new_tokens=50, do_sample=True, temperature=0.7)
print(f"   {caption3}\n")

# 4. Top-p采样
print("4. Top-p采样 (top_p=0.9):")
caption4 = generate_with_params(image, max_new_tokens=50, do_sample=True, top_p=0.9)
print(f"   {caption4}")

## 8. 实际应用示例

### 8.1 辅助视障人士

In [ ]:
def describe_scene_for_accessibility(image):
    """为视障人士描述场景"""
    # 生成详细描述
    description = generate_caption(image, prompt="Describe this image in detail for a blind person:")
    
    # 回答关键问题
    safety_check = visual_qa(image, "Are there any safety hazards visible?")
    location = visual_qa(image, "What kind of place is this?")
    
    return {
        'description': description,
        'safety': safety_check,
        'location': location
    }

print("♿ 无障碍描述:\n")
result = describe_scene_for_accessibility(image)

print(f"场景描述: {result['description']}")
print(f"地点类型: {result['location']}")
print(f"安全检查: {result['safety']}")

### 8.2 社交媒体自动标题

In [ ]:
def generate_social_media_caption(image):
    """生成社交媒体标题"""
    # 生成创意描述
    caption = generate_caption(image, prompt="A creative and engaging social media caption:")
    
    # 生成标签建议
    tags_prompt = "Question: What are 3 relevant hashtags for this image? Answer:"
    inputs = processor(images=image, text=tags_prompt, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    tags = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    
    return caption, tags

print("📱 社交媒体标题生成:\n")
caption, tags = generate_social_media_caption(image)

print(f"标题: {caption}")
print(f"建议标签: {tags}")

## 总结

本教程演示了BLIP-2的核心功能：

1. **图像描述生成** - 自动生成准确的图像描述
2. **视觉问答** - 回答关于图像的各种问题
3. **多轮对话** - 支持连续的交互式问答
4. **批量处理** - 高效处理多张图像
5. **参数调优** - 通过调整生成参数优化输出
6. **实际应用** - 无障碍辅助、社交媒体等场景

### 练习任务

1. 使用自己的图像测试BLIP-2
2. 尝试不同的提示模板
3. 比较不同生成参数的效果
4. 探索BLIP-2的其他应用场景

### 参考资源

- [BLIP-2论文](https://arxiv.org/abs/2301.12597)
- [BLIP-2 GitHub](https://github.com/salesforce/LAVIS)
- [BLIP-2详解文档](../docs/01-模型调研与选型/06-BLIP2模型详解.md)

🎉 恭喜完成本教程！